# <span id="chap-parallel-simple"></span> Setting up a simple compute cluster

Given the [architecture of IPython parallel computation](parallel-ipython.ipynb), we now need to set up the collection of processes that we need. Let's start with the simplest imaginable compute cluster: a controller controlling a single engine accessed from a single client notebook, all on the same machine.

<div class=figure id=fig_ipython_local_cluster_one>
<div class=figurebody>
<img alt="The simplest compute cluster" src="ipython-local-parallelism-one.svg">
<br>
<span class=caption>The simplest compute cluster. The logical architecture, and therefore the view from software, remains the same as for the general case.</span>
</div>
</div>

It should be obvious that this set-up won't give us any speed-up: we have a single engine, and therefore a single computation going on at any time, and therefore no parallelism. All we're really doing is moving computation out of the notebook and into a compute engine &ndash; but on the *same* processor. However, the important point to remember is that, from the perspective or software running in the notebook, this simple cluster is *identical* to any other. This means we can understand how to develop code in this simple environment and then move, fairly seamlessly, to other, more complex set-ups that *do* give us speed-up. We can also use this simple set-up to debug code we intend to be parallel before moving to the more complicated set-ups that are often much harder to debug.

Actually, things are a bit more complicated than this, even in the simplest case. Since there's a good chance the machine you're using is multicore, we can get speedup by having multiple engines that can run one per core.

<div class=figure id=fig_ipython_local_cluster>
<div class=figurebody>
<img alt="The next-to-simplest compute cluster" src="ipython-local-parallelism.svg">
<br>
<span class=caption>On a muilticore machine we can have multiple engines, one per core.</span>
</div>
</div>

## <span id="sec_parallel_simple_ipcluster"></span> Starting a cluster with `ipcluster`

Let's jump straight in and start this cluster:

In [6]:
%%bash --bg

ipcluster start

Starting job # 3 in a separate thread.


...and that's it. Hard to see what all the fuss is about. Of course it goes downhill from here. When we want to stop the cluster, we can simply use:

In [2]:
%%bash --bg

ipcluster stop

Starting job # 2 in a separate thread.


The `ipcluster` command is a wrapper around a set of more complicated (but more powerful) commands that provide the controller and engine processes from the architecture. 

## <span id="sec_parallel_simple_programming"></span> How to write (and run) programs in parallel

Let's write our first parallel program. As we observed earlier, IPython's computing model is *explicitly parallel*, meaning that we identify what we want to happen in parallel by providing the code needed to make it happen. Therefore we need to change, at least to some extent, the programs we would "normally" (sequentially) write to operate in parallel.

The simplest way to use IPython for parallelism is to make use of the "parallal `map`". The `map` funcion in Python takes a function and applies it to each element of a list, returning a new list consisting of the values that the function returns when applied to each element in turn. This works for any function that takes a single argument. For example we can define a factorial function:

In [3]:
def factorial( n ):
    '''Return the factorial of the number given.
    
    n: the number
    returns: the factorial of that number'''
    
    f = 1
    for i in range(1, n + 1):
        f = f * i
    return f

We can then create a list of numbers and apply `factorial` to each element:

In [4]:
ns = [ 2, 6, 10, 20, 30, 40 ]
map(factorial, ns)

[2,
 720,
 3628800,
 2432902008176640000,
 265252859812191058636308480000000L,
 815915283247897734345611269596115894272000000000L]

`map` is an example of what's called a *higher-order function*: it takes another function (`factorial` in this case) as argument and makes use of it (applying it in turn to all the elements of its other argument). A useful way to think of this is that `map` defines a computational pattern &ndash; "apply a function to each element of a list" &ndash; that is then filled-in with the actual function and list you want to apply the pattern to. The pattern of computation that `map` gives rise to is "the same" in all cases, although the details depend on the function being mapped.

The built-in `map` happens to apply the function sequentially and in order to the list elements, but that's just a detail. We can define a "parallel map" that applies the function to each element of the list in parallel, and (if these applications happened on different cores) the result would come back faster. This is the essence of the simplest form of parallelism supported by IPython, and this is the style we'll use almost exclusively in this book.

We first need to load the appropriate IPython support library: 

In [5]:
from ipyparallel import Client

The `ipyparallel` library provides all the code needed to interact with a compute server. What we're going to do is use the `ipyparallel` code to send our factorial function to the compute server we started earlier. Firstly we need to connect to the cluster, which we do by creating a `Client` object:

In [7]:
client = Client()

The `Client` object has a number of methods and other useful features available. We can check how manay engines are available, for example:

In [9]:
print "Local cluster has {n} engines".format(n = len(client[:]))

Local cluster has 4 engines


The machine I use has four cores, so `ipcluster` has started four `ipengine` processes automatically for me. So we should get speed-up,even in this simplest case!

To work with these engines, we need two things. Firstly, as mentioned above, each engine runs in its own IPython process: it doesn't automatically have access to the code and data running in the notebook. So we need to ensure that we import everything we need into the engines' IPython instances. Our current example is so simple that we don't actually need anything, but suppose we needed `numpy`. We could import it into all the engines in the cluster with one command:

In [10]:
with client[:].sync_imports():
    import numpy

importing numpy on engine(s)


Secondly, we need a way of sending work to the engines. We could do this manually: decide on exactly what function and values we want each engine to work on, and instruct IPython to move the necessary code and data to the appropriate process. There are times when this is a sensible option, when you want complete control: but these cases are rare, and  there's an easier way.

IPython takes a cluster and constructs "views" onto it. There are several different kinds &ndash; the view that gives you complete control is called a *direct* view &ndash; but for our purposes we'll make use of a different kind: a *load-balanced* view. The idea is that "jobs" of work are submitted to the load-balanced view, and the view itself handles how to allocate jobs to engines. There are several issues with this that we'll return to later, but for the time being we'll simply obtain a load-balanced view from the client object: 

In [12]:
view = client.load_balanced_view()

The view provides operations for sending work to the cluster. In particular, it includes the "parallel map" function that applies a function to elements of a list in parallel. What does this mean? The map function will send each engine an element of the array and the function to apply to it. It will collect all the results as they come back and assemble them into a result list, just like the built-in `map` function. If there are more elements in the list than there are engines in the cluster (which will normally be the case) it will get all the engines working and, when an engine finishes computing for one element, it will send it another. This is what load balancing means: the view creates a set of jobs for the parallel map, where each job applied the function to a single list element, and sends them to a free engine. The parallel map finishes when all the individual jobs have finished, and the jobs run in parallel to the extent that there are free engines available.

Putting this together we can perform our factorial map in parallel:

In [13]:
view.map_sync(factorial, ns)

[2,
 720,
 3628800,
 2432902008176640000,
 265252859812191058636308480000000L,
 815915283247897734345611269596115894272000000000L]

This is identical to the result above, which is the point: built-in `map` and `map_sync` on the view are essentially identical from the programmer's perspective, with the latter being faster *if* there's a lot of work to be done and *if* there are multiple cores on which to do it. We could apply a much more complicated function in exactly the same way and have it automatically farmed-out to parallel engines. This is explicitly parallel in the sense that we've instructed IPython to use parallelism on the cluster: *we* made the decision, not IPython. However, IPython's parallelism library takes care of all the mechanics of dividing-up the map into a collection of parallel jobs and sending them to the cluster, thereby saving us a lot of work and letting the *same* code work regardless of the number of engines we happen to have available.     

## <span id="sec_parallel_simple_cluster_mechanics"></span> How IPython clusters work

To properly understand what's happening here, it's worth understanding how the cluster is going to work under the bonnet, at least at a superficial level. (If you're really sure you don't care, feel free to skip this section.)

We've mentioned a couple of times that each engine (and the notebook, and the controller) are running in separate IPython processes, which implies that all code and data either have to be imported into the interpreter or passed to them somehow. We saw the former process happening above with `sync_imports`; the latter is hidden inside the `map_sync` call.

<div class=figure id=fig_ipython_mechanics>
<div class=figurebody>
<img alt="IPython message exchange" src="ipython-mechanics.svg">
<br>
<span class=caption>The mechanics of IPython parallelism.</span>
</div>
</div>

When we call `map_sync`, IPython generates a number of jobs, one per list element. The jobs consist of applying the function being mapped to one list element, and each one is independent of the others and can be executed in parallel. Each job is given a *message id* that will identify it within the system, and is sent to the cluster controller. The controller queues-up the jobs and sends them, one at a time, to a free engine. As engines finish their jobs, the controller saves the results against the corresponding message id. Once all the message ids have results associated with them &ndash; that is, all the component jobs have been executed successfully to completion &ndash; the controller returns these results to the notebook. The notebook assembles the results into a result list in the right order and returns them to the user program.

The same cluster might be being used by several notebooks, so the controller might be dealing with several independent streams of jobs. Engines might be of different speeds, and so some jobs might complete more quickly than others. Or the jobs themselves might be of different sizes, some much easier than others. *None of that matters.* The view and controller will take care of the mechanics. As far as the programmer is concerned, calling `map_sync` causes some computation to occur exactly as would happen when calling `map`, but hopefully faster.